# Access and plot LLC4320 from Google Cloud

#### Most of the code used here was patched together from examples on the Pangeo website. I think Ryan Abernathy can be credited for most of it. <br>Don't worry about understanding the steps to get the data!!

In [1]:
# import some packages
import random
import os
import xarray as xr
import numpy as np
import xgcm # this is an xarray-based package to deal with GCM output
from matplotlib import pyplot as plt
from matplotlib import image
import cmocean # this is a colormap package
import cartopy.crs as ccrs # this is a mapping package
%matplotlib inline

#### Some of the steps below might require setting up a Google cloud account. If that's the case, I'll help you do that. 
I think there are instructions how to do that [here.](https://catalog.pangeo.io/browse/master/ocean/LLC4320/LLC4320_SSH)

In [4]:
from intake import open_catalog # this is a convenience package for data i/o
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")
list(cat)

['LLC4320_grid',
 'LLC4320_SST',
 'LLC4320_SSS',
 'LLC4320_SSH',
 'LLC4320_SSU',
 'LLC4320_SSV']

#### Since the model files are huge, we are loading the ONLY THE GRID into a dask file (that's like an xarray data structure but can be used to parallelize):

In [5]:
ds = cat.LLC4320_grid.to_dask()
ds

ValueError: No plugins loaded for this entry: zarr
A listing of installable plugins can be found at https://intake.readthedocs.io/en/latest/plugin-directory.html .

In [ ]:
# trick to make things run faster:
coords = ds.coords.to_dataset().reset_coords()
ds = ds.reset_coords(drop=True)

### Note on data structure:
The MITgcm LLC3420 have the following structure:
There are 13 "tiles" which are pretty much 3d boxes.<br> Each tiles has latitude-longitude-depth coordinates with 4320x4320x90. <br>
Over the whole globe that works out to be a **2km horizontal grid spacing**.

Chech out this [lower resolution model](https://ecco-v4-python-tutorial.readthedocs.io/fields.html) that uses the same tiles. **Taiwan is in tile 5**.

The model time from start to end is roughly one year with hourly output.

In [ ]:
# this is what the grid structure looks like
# i and i_g are indices for the longitude dimension (i is at the cell center and i_g at the side)
# j and j_g are indices for the latitude dimension
# k is for the vertical (but doesn't pertain us here)
ds

## Slicing the data
Since the data file are big, we only choose a small subset. We do that by using xarray slicing techniques. <br>**ntile** is the tile we pick,<br> **nt** is the time,<br> **i_slice** and **j_slice** are the ranges of longitude and latitude data, those are indices from the 4320x4320 grid.

In [ ]:
ntime = 100 # time picked at random
ntile = 5 # tile 5 is for Taiwan
i_slice = slice(2750,4320) # I picked those such that taiwan is in the focus and land is cut off
j_slice =slice(0,2000)

selector = dict(time=ntime, face=ntile, i=i_slice, j=j_slice,
                i_g=i_slice, j_g=j_slice)
grid_ds = grid_full.isel(**selector)

In [ ]:
# range of longitudes
grid_ds.XC.values.min(),grid_ds.XC.values.max()

In [ ]:
# range of latitudes
grid_ds.YC.values.min(),grid_ds.YC.values.max()

In [ ]:
# where the depth is zero there is likely land
grid_ds.Depth.plot()

### After defining the region, we can now read the data variables:

**SSH** is sea surface height, <br>
**SST** is sea surface temperature. <br>
**SSS** is sea surface salinity.<br>
**(u,v)** are the horizontal velocity

In [ ]:
ssh = cat.LLC4320_SSH(chunks=False).to_dask().isel(time=ntime, face=ntile, i=i_slice, j=j_slice)
sst = cat.LLC4320_SST(chunks=False).to_dask().isel(time=ntime, face=ntile, i=i_slice, j=j_slice)
# sss has some extra metadata
sss = cat.LLC4320_SSS(chunks=False).to_dask().isel(time=ntime, face=ntile, i=i_slice, j=j_slice)[['SSS']].reset_coords(drop=True)
u = cat.LLC4320_SSU(chunks=False).to_dask().isel(time=ntime, face=ntile, i_g=i_slice, j=j_slice)
v = cat.LLC4320_SSV(chunks=False).to_dask().isel(time=ntime, face=ntile, i=i_slice, j_g=j_slice)
ds = xr.merge([grid_ds, ssh, sst, sss, u, v])

# vertical coordiantes are not helpful
ds = ds.drop(['Z', 'Zl', 'Zp1', 'Zu', 'k', 'k_l', 'k_p1', 'PHrefF', 'drC'])
ds

In [ ]:
# make sure that longitudes are between -180 and 180
def maybe_wrap_lon(lon):
    if abs(lon.min() - lon.max()) < 180:
        return lon
    else:
        return lon.where(lon > 0, other=(lon + 360))
    
ds.coords['XC'] = maybe_wrap_lon(ds.XC)
ds.coords['YG'] = maybe_wrap_lon(ds.YG)

In [ ]:
# this is prep work for the map
# finding the center of the selected map

central_lon = ds.XC.mean().values.item()
central_lat = ds.YC.mean().values.item()

lon_min = ds.XC.min().values.item()
lon_max = ds.XC.max().values.item()
lon_range = lon_max - lon_min

lat_min = ds.YC.min().values.item()
lat_max = ds.YC.max().values.item()
lat_range = lat_max - lat_min

proj = ccrs.Orthographic(central_longitude=central_lon,
                         central_latitude=central_lat)

date_str = np.datetime_as_string(ds.time.values, timezone='UTC', unit='m')

location = f'{central_lon:3.1f}, {central_lat:3.1f} | {date_str}'
print(location)

### The grids are very complicated, so there is a tool that deals with this. 

In [ ]:
grid = xgcm.Grid(ds, periodic=False)
grid

In [ ]:
# Don't worry about these calculations yet. They will be useful later on.
# EKE is eddy kinetic energy
# Zeta is vorticity
# Div is divergence
ds['eke'] = 0.5 * (grid.interp(ds.U**2, 'X', boundary='extend')
             + grid.interp(ds.V**2, 'Y', boundary='extend'))

ds['zeta'] = 1e4 * (-grid.diff(ds.U * ds.dxC, 'Y', boundary='extend') +
                    grid.diff(ds.V * ds.dyC, 'X', boundary='extend'))/ds.rAz

ds['div'] = (grid.diff(ds.U * ds.dxC, 'X', boundary='extend') +
             grid.diff(ds.V * ds.dyC, 'Y', boundary='extend'))/ds.rA

In [ ]:
# fix some metadata for plotting
ds.zeta.attrs['units'] = r'$10^{-4}$ s$^{-1}$'
ds.zeta.attrs['long_name'] = 'Vorticity'

ds.SST.attrs['units'] = r'$^\circ$C'
ds.SST.attrs['long_name'] = 'Sea Surface Temperature'

ds.SSS.attrs['units'] = r'psu'
ds.SSS.attrs['long_name'] = 'Sea Surface Salinity'

ds.Eta.attrs['units'] = r'm'
ds.Eta.attrs['long_name'] = 'Sea Surface Height'

In [ ]:
# lon_min near equator are square
# towards pole, both dimensions contract
scale_lon = 2 + 0.75 * abs(np.deg2rad(central_lat))
scale_lat = 2 + 0.3 * abs(np.deg2rad(central_lat))

print(scale_lon, scale_lat)

extent = [central_lon - lon_range/scale_lon, central_lon + lon_range/scale_lon,
          central_lat - lat_range/scale_lat, central_lat + lat_range/scale_lat]

### This is a manual plotting function to set up the map.
The main point here is that we are plotting spherial data onto a 2D grid, so we make to make a choice about how to project that. Like in every map of the Earth.

In [ ]:
plt.rcParams['font.size'] = 16
def plot(da, clip_extent=True, **kwargs):
    xdim = 'XC' if 'i' in da.dims else 'XG'
    ydim = 'YC' if 'j' in da.dims else 'YG'
    
    fig = plt.figure(figsize=(13, 10))
    ax = fig.add_axes([0, 0.02, 1, 0.91],
                      projection=ccrs.Orthographic(central_lon, central_lat))
    ax.background_patch.set_facecolor('0.6')
    if clip_extent:
        ax.set_extent(extent, crs=ccrs.PlateCarree())
    gl = ax.gridlines()
    
    da.plot(ax=ax, x=xdim, y=ydim, transform=ccrs.PlateCarree(), **kwargs)
    ax.set_title(f'LLC4320 {da.long_name} | {location}')

In [ ]:
plot(ds.Depth, center=False, robust=True, cmap=cmocean.cm.thermal,
     cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)
plt.savefig('DEPTH.png')

In [ ]:
plot(ds.SST, center=False, robust=True, cmap=cmocean.cm.thermal,
     cbar_kwargs={'pad':0.01,'aspect': 30}, rasterized=True)
plt.savefig('SST.png')

In [ ]:
plot(ds.SSS, center=False, robust=True, cmap=cmocean.cm.haline,
     cbar_kwargs={'pad':0.01,'aspect': 30})
plt.savefig('SSS.png')

In [ ]:
plot(ds.Eta, center=False, robust=True, cmap=cmocean.cm.dense_r,
     cbar_kwargs={'pad':0.01,'aspect': 30})
plt.savefig('SSH.png')

In [ ]:
plot(ds.zeta, robust=True, cmap=cmocean.cm.curl, cbar_kwargs={'pad':0.01,'aspect': 30})
plt.savefig('Vort.png')